In [1]:
import numpy as np
import json

In [2]:
class Job:
    def __init__(self, jid, Sj, rj, dj, wj):
        self.id = jid # job id
        self.S = Sj   # sequence of id of Tasks (list)
        self.r = rj   # release date
        self.d = dj   # due date
        self.w = wj   # weight
    
    def __str__(self):
        return f"id={self.id:>3}  w={self.w:>2}  r:{self.r:>3}  d:{self.d:>3}  " + \
               f"min_duration={self.min_duration():>3}  B:{self.B():>3}  " + \
               f"C:{self.C():>3}  late?{self.C()>self.d:>1}  delay={max(0, self.C()-self.d):>2}  cost={self.cost():>4}"
    
    def min_duration(self):
        return np.sum([tasks[tid].p for tid in self.S])
    
    def min_remaining_duration(self):
        return np.sum([tasks[tid].p for tid in self.S if not tasks[tid].done])
    
    def B(self):
        return tasks[self.S[0]].B # beginning date
    
    def C(self):
        return tasks[self.S[-1]].C # completion date
    
    def cost(self):
        C = self.C()
        T = max(C - self.d, 0)
        U = 1 if T > 0 else 0
        return self.w * (C + alpha*U + beta*T)

class Task:
    def __init__(self, tid, jid, pi, workers):
        self.id = tid          # Task id
        self.jid = jid         # Job id of corresponding Job
        self.p = pi            # processing time
        self.workers = workers # possible Workers for this Task (list)
        self.B = None          # beginning date
        self.C = None          # completion date
        self.running = False
        self.done = False
        self.mid = None        # id of Machine on which task runs
        self.oid = None        # id of Operator doing this task
        
    def __str__(self):
        j = jobs[self.jid]
        delay = int(self.B - (j.r + np.sum([tasks[tid2].p for tid2 in jobs[self.jid].S if tid2 < self.id])))
        return f"id={self.id:>3}  job={self.jid:>3}  begin:{self.B:>3}  complete:{self.C:>3}  " + \
               f"(time:{self.p:>2})  (delay:{delay:>2})  " + \
               f"machine:{self.mid:>2}  operator:{self.oid:>2}"
        
class Worker:
    def __init__(self, mid, oid):
        self.mid = mid # id of Machine used
        self.oid = oid # id of Operator working
        
    def __str__(self):
        return f"machine={self.mid} operator:{self.oid}"
    
class WorkUnit:
    def __init__(self, time, jid, t_idx, tid, w):
        self.time = time   # time at which WorkUnit is considered to start
        self.jid = jid     # Job id
        self.t_idx = t_idx # index of Task in Job's Tasks sequence
        self.tid = tid     # Task id
        self.w = w         # Worker

In [3]:
def parser_job(job):
    jid = job['job']
    Sj  = job['sequence']
    rj  = job['release_date']
    dj  = job['due_date']
    wj  = job['weight']
    return Job(jid, Sj, rj, dj, wj)

def parser_task(task, jid):
    tid = task['task']
    pi  = task['processing_time']
    workers = []
    for machine in task['machines']:
        mid = machine['machine']
        for operator in machine['operators']:
            oid = operator
            workers.append(Worker(mid, oid))
    return Task(tid, jid, pi, workers)

def parser_inst(inst):
    J = inst['parameters']['size']['nb_jobs']
    I = inst['parameters']['size']['nb_tasks']
    M = inst['parameters']['size']['nb_machines']
    O = inst['parameters']['size']['nb_operators']
    alpha = inst['parameters']['costs']['unit_penalty']
    beta = inst['parameters']['costs']['tardiness']
    
    machines  = {} # storing machines' availability by id
    operators = {} # storing operators' availability by id
    jobs  = (J+1) * [None]                          # list of Jobs
    jobs[0] = Job(jid=0, Sj=[], rj=-1, dj=-1, wj=0) # dummy element 0
    tasks = (I+1) * [None]                          # list of Tasks
    tasks[0] = Task(tid=0, jid=0, pi=0, workers=[]) # dummy element 0
    for job in inst['jobs']:
        jid = job['job']
        jobs[jid] = parser_job(job) # add Job in list of Jobs
        for task in inst['tasks']:
            tid = task['task']
            if tid in jobs[jid].S:
                tasks[tid] = parser_task(task, jid) # add task in list of Tasks
                for machine in task['machines']:
                    mid = machine['machine']
                    for operator in machine['operators']:
                        oid = operator
                        machines[mid]  = True # set machine to available
                        operators[oid] = True # set operator to available
    
    return (J, I, M, O, alpha, beta, jobs, tasks, machines, operators)

In [4]:
def optim():
    time = 0
    while (np.sum([t.done for t in tasks]) < I):
        for t in tasks:
            if t.running:
                if (time - t.B >= t.p):
                    t.running = False # task ends
                    t.done = True     # task is done
                    machines[t.mid]  = True # free machine
                    operators[t.oid] = True # free operator
                    t.C = time              # set completion time
        
        for j in jobs:
            if time >= j.r: # it is past the release date of this Job
                for (t_idx, tid) in enumerate(j.S): # loop over Tasks for this Job
                    t = tasks[tid]      # get Task from its id
                    for w in t.workers: # look for a Worker (machine, operator) to execute Task
                        if operators[w.oid] \
                        and machines[w.mid] \
                        and not t.running \
                        and not t.done \
                        and np.all([tasks[tid2].done for tid2 in j.S[:t_idx]]): # all Job's previous Tasks are done
                            t.running = True # set task to running
                            t.B = time       # set beginning time
                            t.mid = w.mid    # set machine id for task
                            t.oid = w.oid    # set operator id for task
                            machines[w.mid]  = False   # set machine to busy
                            operators[w.oid] = False   # set operator to busy

        time += 1 # time flows

In [5]:
def optim2():
    
    ########## SUBROUTINE ##########
    def get_admissible_work_units():
        work_units = []
        for j in jobs:
            if time >= j.r: # it is past the release date of this Job
                for (t_idx, tid) in enumerate(j.S): # loop over Tasks for this Job
                    t = tasks[tid] # get Task from its id
                    if (not t.running) \
                    and (not t.done) \
                    and np.all([tasks[tid2].done for tid2 in j.S[:t_idx]]): # all Job's previous Tasks are done
                        for w in t.workers: # look for a Worker (machine, operator) to execute Task
                            if operators[w.oid] and machines[w.mid]: # Worker is free
                                work_units.append(WorkUnit(time, j.id, t_idx, tid, w))
        return work_units
    ################################
    
    time = 0
    while (np.sum([t.done for t in tasks]) < I):
        
        ################# UPDATE RUNNING TASKS ##################
        for t in tasks:
            if t.running:
                if (time - t.B >= t.p):
                    t.running = False # task ends
                    t.done = True     # task is done
                    machines[t.mid]  = True # free machine
                    operators[t.oid] = True # free operator
                    t.C = time              # set completion time
        #########################################################
        
        work_units = get_admissible_work_units()
        while len(work_units) > 0: # while we can still start tasks
            work_units = sorted(work_units, key=WU_weight)
            chosen_work_unit = work_units[-1] # choose the work unit with the biggest weight to finish it at the earliest
            t = tasks[chosen_work_unit.tid]
            w = chosen_work_unit.w
            t.running = True # set task to running
            t.B = time       # set beginning time
            t.mid = w.mid    # set machine id for task
            t.oid = w.oid    # set operator id for task
            machines[w.mid]  = False   # set machine to busy
            operators[w.oid] = False   # set operator to busy
            
            work_units = get_admissible_work_units()
                    
        time += 1 # time flows

In [6]:
def cost(jobs):
    s = 0
    for job in jobs[1:]:
        C = job.C()
        T = max(C - job.d, 0)
        U = 1 if T > 0 else 0
        s += job.w * (C + alpha*U + beta*T)
    return s

In [7]:
def jsonify(tasks):
    res = []
    for task in tasks[1:]:
        sub_res = {}
        sub_res['task'], sub_res['start'], sub_res['machine'], sub_res['operator'] = task.id, task.B, task.mid, task.oid
        res.append(sub_res)
    return res

In [8]:
def WU_weight(work_unit):
    j = jobs[work_unit.jid]  # corresponding Job
    t = tasks[work_unit.tid] # corresponding Task
    C = work_unit.time + j.min_remaining_duration() # proxy for job completion time
    T = max(C - j.d, 0)
    U = 1 if T > 0 else 0
    return w1*j.w*C + w2*j.w*j.d + w3*j.w*alpha*T + w4*j.w*beta*U

---

In [9]:
in_filename = "instances/huge.json"
with open(in_filename, 'rb') as f:
    inst = json.load(f)

In [10]:
(J, I, M, O, alpha, beta, jobs, tasks, machines, operators) = parser_inst(inst)
print(f"J={J}\nI={I}\nM={M}\nO={O}\nα={alpha}\nβ={beta}")

J=200
I=1000
M=50
O=30
α=6
β=1


In [11]:
(J, I, M, O, alpha, beta, jobs, tasks, machines, operators) = parser_inst(inst)
w1, w2, w3, w4 = 1, -1, 0, 0 # default weights, working OK
optim2()
cost(jobs)

47194

In [12]:
for j in sorted(jobs[1:], key=lambda j: -j.cost())[:20]:
    print(j)

id=141  w=16  r:  9  d:116  min_duration= 16  B: 41  C: 60  late?0  delay= 0  cost= 960
id=  9  w=16  r:  3  d:132  min_duration= 16  B: 43  C: 59  late?0  delay= 0  cost= 944
id=168  w=15  r:  2  d: 90  min_duration= 15  B: 37  C: 52  late?0  delay= 0  cost= 780
id=198  w=15  r: 10  d: 96  min_duration= 15  B: 37  C: 52  late?0  delay= 0  cost= 780
id= 55  w=13  r:  9  d:114  min_duration= 13  B: 39  C: 54  late?0  delay= 0  cost= 702
id= 92  w=12  r:  2  d:118  min_duration= 12  B: 41  C: 58  late?0  delay= 0  cost= 696
id= 60  w=12  r:  9  d:149  min_duration= 12  B: 44  C: 56  late?0  delay= 0  cost= 672
id=112  w=12  r:  1  d:149  min_duration= 12  B:  1  C: 56  late?0  delay= 0  cost= 672
id= 24  w=13  r:  6  d: 81  min_duration= 13  B: 35  C: 51  late?0  delay= 0  cost= 663
id=144  w=12  r:  4  d:149  min_duration= 12  B: 42  C: 55  late?0  delay= 0  cost= 660
id=114  w=12  r:  8  d: 86  min_duration= 12  B: 41  C: 54  late?0  delay= 0  cost= 648
id=180  w=12  r:  6  d:132  min_

In [13]:
for t in sorted(tasks[1:], key=lambda t: (t.jid, t.B))[:20]:
    print(t)

id=  1  job=  1  begin:  2  complete:  4  (time: 2)  (delay: 0)  machine:33  operator:21
id=344  job=  1  begin:  4  complete:  5  (time: 1)  (delay: 0)  machine:43  operator:25
id=377  job=  1  begin:  5  complete:  7  (time: 2)  (delay: 0)  machine:26  operator:25
id=498  job=  1  begin:  7  complete:  8  (time: 1)  (delay: 0)  machine:47  operator:27
id=512  job=  1  begin:  9  complete: 10  (time: 1)  (delay: 1)  machine:37  operator:27
id=513  job=  1  begin: 10  complete: 11  (time: 1)  (delay: 1)  machine: 6  operator:21
id=  2  job=  2  begin: 20  complete: 21  (time: 1)  (delay:16)  machine:26  operator: 1
id=256  job=  2  begin: 25  complete: 26  (time: 1)  (delay:20)  machine:38  operator:16
id=284  job=  2  begin: 26  complete: 28  (time: 2)  (delay:20)  machine:19  operator:16
id=510  job=  2  begin: 28  complete: 29  (time: 1)  (delay:20)  machine:28  operator:25
id=771  job=  2  begin: 29  complete: 30  (time: 1)  (delay:20)  machine:43  operator:20
id=  3  job=  3  begi

---

In [14]:
out_filename = "solutions/KIRO-huge.json"
with open(out_filename, 'w') as f:
    json.dump(jsonify(tasks), f)